# Clone data and dependencies

In [ ]:
!pip install underthesea
!pip install nltk
!pip install transformers
!pip install cython
!pip install youtokentome
!pip install datetime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 12.6 MB/s 
     |████████████████████████████████| 235 kB 50.1 MB/s 
     |████████████████████████████████| 581 kB 59.9 MB/s 
     |████████████████████████████████| 965 kB 55.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 29.9 MB/s 
     |████████████████████████████████| 101 kB 14.0 MB/s 
     |████████████████████████████████| 596 kB 66.4 MB/s 
     |████████████████████████████████| 6.6 MB 45.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
%cd /gdrive/My\ Drive/Group\ 3 - NLP\ Project/source_code

/gdrive/My Drive/NLP_Project


In [ ]:
from src.vocabulary import Vocabulary
from src.dataset import StanfordEnViDataset,get_loader
from src.model import EnViTransformer
from src.utils import save_checkpoint, load_checkpoint, training_step, validate, beam_search
from src.Translator import TransformerTranslator
import numpy as np
import random
import youtokentome as yttm
import os
from src.dataset import StanfordEnViDataset,get_loader, add_upper_token


In [ ]:
import yaml
from yaml.loader import SafeLoader
import torch
import datetime
with open('./utils/config/contractions_config.yml') as f:
  contractions_config = yaml.load(f, Loader=SafeLoader)

print(contractions_config)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

contractions_Translator = TransformerTranslator(contractions_config, device, input_type = 'word_segmentation')
print(len(contractions_Translator.vi_vocab), len(contractions_Translator.en_vocab))


{'data': {'train': {'src_file': './data/processed/augmented/contractions/en_train_contractions_full.txt', 'tgt_file': './data/processed/augmented/contractions/vi_train_contractions_full.txt'}, 'dev': {'src_file': './data/processed/dev/tst2012.en', 'tgt_file': './data/processed/dev/tst2012.vi'}}, 'utils': {'vocab': {'en': './utils/vocab/contractions/en.json', 'vi': './utils/vocab/contractions/vi.json'}, 'training_runs': {'train_loss': './utils/runs/contractions/train_loss', 'dev_loss': './utils/runs/contractions/dev_loss', 'learning_rate': './utils/runs/contractions/learning_rate'}}, 'train': {'num_epochs': 20, 'learning_rate': 0.001, 'batch_size': 64}, 'model': {'max_sent_len': 120, 'embedding_size': 512, 'num_heads': 8, 'num_encoder_layers': 3, 'num_decoder_layers': 3, 'dropout': 0.1, 'forward_expansion': 4, 'checkpoint_path': './model/contractions_model.pth.tar'}}
19438 26641


In [ ]:
contractions_Translator.en_vocab.stoi

{'<PAD>': 0,
 '<SOS>': 1,
 '<EOS>': 2,
 '<UNK>': 3,
 '<upper>': 4,
 'rachel': 5,
 'pike': 6,
 ':': 7,
 'the': 8,
 'science': 9,
 'behind': 10,
 'a': 11,
 'climate': 12,
 'headline': 13,
 'in': 14,
 '4': 15,
 'minutes': 16,
 ',': 17,
 'atmospheric': 18,
 'chemist': 19,
 'provides': 20,
 'glimpse': 21,
 'of': 22,
 'massive': 23,
 'scientific': 24,
 'effort': 25,
 'bold': 26,
 'headlines': 27,
 'on': 28,
 'change': 29,
 'with': 30,
 'her': 31,
 'team': 32,
 '--': 33,
 'one': 34,
 'thousands': 35,
 'who': 36,
 'contributed': 37,
 'taking': 38,
 'risky': 39,
 'flight': 40,
 'over': 41,
 'rainforest': 42,
 'pursuit': 43,
 'data': 44,
 'key': 45,
 'molecule': 46,
 '.': 47,
 'i': 48,
 "'d": 49,
 'like': 50,
 'to': 51,
 'talk': 52,
 'you': 53,
 'today': 54,
 'about': 55,
 'scale': 56,
 'that': 57,
 'goes': 58,
 'into': 59,
 'making': 60,
 'see': 61,
 'paper': 62,
 'look': 63,
 'this': 64,
 'when': 65,
 'they': 66,
 'have': 67,
 'do': 68,
 'and': 69,
 'air': 70,
 'quality': 71,
 'or': 72,
 'smog

In [ ]:
contractions_Translator.model

EnViTransformer(
  (src_word_embedding): Embedding(26641, 512)
  (src_position_embedding): Embedding(120, 512)
  (tgt_word_embedding): Embedding(19438, 512)
  (tgt_position_embedding): Embedding(120, 512)
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
          )
          (linear1): Linear(in_features=512, out_features=4, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=4, out_features=512, bias=True)
          (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
        (1): TransformerEncoderLayer(
          (sel

In [ ]:
contractions_Translator.train()

**************************************************************************    Epoch number 1    **************************************************************************


  0%|          | 0/2810 [00:00<?, ?it/s]

Current step: 1, epoch: 1    Training loss:  10.155844688415527    Evaluation loss:  10.161013336181641    Time elapsed:  0:00:06.476371    (Best weights saved!) 
Current step: 250, epoch: 1    Training loss:  8.875734239578247    Evaluation loss:  7.5364474678039555    Time elapsed:  0:01:42.899985    (Best weights saved!) 
Current step: 500, epoch: 1    Training loss:  7.814539597511292    Evaluation loss:  6.3556305694580075    Time elapsed:  0:03:21.051850    (Best weights saved!) 
Current step: 750, epoch: 1    Training loss:  7.309922949473063    Evaluation loss:  6.0894606399536135    Time elapsed:  0:04:59.579560    (Best weights saved!) 
Current step: 1000, epoch: 1    Training loss:  7.001101315021515    Evaluation loss:  5.846527309417724    Time elapsed:  0:06:37.907277    (Best weights saved!) 
Current step: 1250, epoch: 1    Training loss:  6.771181003570557    Evaluation loss:  5.620418014526368    Time elapsed:  0:08:16.267213    (Best weights saved!) 
Current step: 150

  0%|          | 0/2810 [00:00<?, ?it/s]

Current step: 3000, epoch: 2    Training loss:  4.917151837599905    Evaluation loss:  4.819979286193847    Time elapsed:  0:19:44.362079    (Best weights saved!) 
Current step: 3250, epoch: 2    Training loss:  4.880104700001803    Evaluation loss:  4.752519626617431    Time elapsed:  0:21:22.717880    (Best weights saved!) 
Current step: 3500, epoch: 2    Training loss:  4.845484180726867    Evaluation loss:  4.668589916229248    Time elapsed:  0:23:01.038893    (Best weights saved!) 
Current step: 3750, epoch: 2    Training loss:  4.811426353961863    Evaluation loss:  4.614457664489746    Time elapsed:  0:24:39.319438    (Best weights saved!) 
Current step: 4000, epoch: 2    Training loss:  4.778996459175558    Evaluation loss:  4.550039749145508    Time elapsed:  0:26:17.450290    (Best weights saved!) 
Current step: 4250, epoch: 2    Training loss:  4.7443935506873665    Evaluation loss:  4.478607978820801    Time elapsed:  0:27:55.581720    (Best weights saved!) 
Current step: 4

...

  0%|          | 0/2810 [00:00<?, ?it/s]

Current step: 25500, epoch: 10    Training loss:  3.1166747535978043    Evaluation loss:  3.600961685180664    Time elapsed:  2:46:20.349252    
Current step: 25750, epoch: 10    Training loss:  3.1283820271492004    Evaluation loss:  3.6081779861450194    Time elapsed:  2:47:56.805793    
Current step: 26000, epoch: 10    Training loss:  3.134193311946493    Evaluation loss:  3.603917236328125    Time elapsed:  2:49:33.439619    
Current step: 26250, epoch: 10    Training loss:  3.141488673786322    Evaluation loss:  3.6132133197784424    Time elapsed:  2:51:09.955065    
Current step: 26500, epoch: 10    Training loss:  3.144599103730572    Evaluation loss:  3.616293315887451    Time elapsed:  2:52:46.482059    
Current step: 26750, epoch: 10    Training loss:  3.1484387456554255    Evaluation loss:  3.6217995357513426    Time elapsed:  2:54:22.997683    
Current step: 27000, epoch: 10    Training loss:  3.1506645217973586    Evaluation loss:  3.6101270866394044    Time elapsed:  2:5

  0%|          | 0/2810 [00:00<?, ?it/s]

Current step: 28250, epoch: 11    Training loss:  3.045569208463033    Evaluation loss:  3.5913251495361327    Time elapsed:  3:04:08.554268    
Current step: 28500, epoch: 11    Training loss:  3.052228336930275    Evaluation loss:  3.5910181999206543    Time elapsed:  3:05:45.101686    
Current step: 28750, epoch: 11    Training loss:  3.0580585428384635    Evaluation loss:  3.5940556812286375    Time elapsed:  3:07:21.627918    
Current step: 29000, epoch: 11    Training loss:  3.066019817988078    Evaluation loss:  3.593781690597534    Time elapsed:  3:08:58.167491    
Current step: 29250, epoch: 11    Training loss:  3.07029853489088    Evaluation loss:  3.588062953948975    Time elapsed:  3:10:34.679659    
Current step: 29500, epoch: 11    Training loss:  3.074284699644361    Evaluation loss:  3.587861318588257    Time elapsed:  3:12:11.167157    
Current step: 29750, epoch: 11    Training loss:  3.0776795424837053    Evaluation loss:  3.5856642436981203    Time elapsed:  3:13:4

  0%|          | 0/2810 [00:00<?, ?it/s]

In [ ]:
bpeTranslator.inference(['I am a student at Cambridge University', 'My family was not poor , and myself , I had never experienced hunger since 2002 .'
, 'Kim Jong Un is the president of Korea'])

=> Loading checkpoint


[' Tôi là một sinh viên ở đại học Cambridge Cambridge. ',
 ' Gia đình tôi không nghèo , và tôi đã không bao giờ trải qua sự đói kém từ năm 2002. ',
 ' Kim Jorgan là tổng thống của Hàn Quốc ']

In [ ]:
import underthesea
en_test_sents = []
with open("./data/processed/test/tst2013.en") as f:
    for sent in f:
        sent  = sent.strip()
        en_test_sents.append(sent)

vi_test_sents = []
with open("./data/processed/test/tst2013.vi") as f:
    for sent in f:
        sent  = sent.strip()
        if sent[-2:] == " .":
            sent = sent[:-2] + "."
        if sent[-2:] == " ?":
            sent = sent[:-2] + "?"
        if sent[-2:] == " !":
            sent = sent[:-2] + "!"

        vi_test_sents.append(sent)


In [ ]:
translations = bpeTranslator.inference(en_test_sents)

=> Loading checkpoint


In [ ]:
vi_test_sents[0:10]

['Khi tôi còn nhỏ , Tôi nghĩ rằng BắcTriều Tiên là đất nước tốt nhất trên thế giới và tôi thường hát bài " Chúng ta chẳng có gì phải ghen tị . "',
 'Tôi đã rất tự hào về đất nước tôi.',
 'Ở trường , chúng tôi dành rất nhiều thời gian để học về cuộc đời của chủ tịch Kim II- Sung , nhưng lại không học nhiều về thế giới bên ngoài , ngoại trừ việc Hoa Kỳ , Hàn Quốc và Nhật Bản là kẻ thù của chúng tôi.',
 'Mặc dù tôi đã từng tự hỏi không biết thế giới bên ngoài kia như thế nào , nhưng tôi vẫn nghĩ rằng mình sẽ sống cả cuộc đời ở BắcTriều Tiên , cho tới khi tất cả mọi thứ đột nhiên thay đổi.',
 'Khi tôi lên 7 , tôi chứng kiến cảnh người ta xử bắn công khai lần đầu tiên trong đời , nhưng tôi vẫn nghĩ cuộc sống của mình ở đây là hoàn toàn bình thường.',
 'Gia đình của tôi không nghèo , và bản thân tôi thì chưa từng phải chịu đói.',
 'Nhưng vào một ngày của năm 1995 , mẹ tôi mang về nhà một lá thư từ một người chị em cùng chỗ làm với mẹ.',
 'Trong đó có viết : Khi chị đọc được những dòng này th

In [ ]:
translations[0:10]

[' Khi tôi còn nhỏ , tôi nghĩ đất nước mình là những người tốt nhất trên hành tinh , và tôi lớn lên một bài hát có tên là " Không đến Envy " ',
 ' Và tôi rất tự hào. ',
 ' Trong trường học , chúng tôi dành rất nhiều thời gian nghiên cứu về lịch sử của Kim <upper>sl , nhưng chúng tôi chưa bao giờ học nhiều về thế giới bên ngoài , ngoại trừ Hàn Quốc , Nhật Bản là kẻ thù của Kiều thốn. ',
 ' Mặc dù tôi vẫn băn khoăn về thế giới bên ngoài , tôi nghĩ rằng mình sẽ dành cả đời sống ở Bắc Triều tiên , cho đến mọi thứ đột nhiên thay đổi. ',
 ' Khi tôi 7 tuổi , tôi đã nhìn thấy sự phân ly công chúng đầu tiên , nhưng tôi nghĩ cuộc sống của mình ở Bắc Triều tiên là bình thường. ',
 ' Gia đình tôi không nghèo , và chính tôi , tôi chưa từng gặp nạn đói. ',
 ' Nhưng một ngày , vào năm 1995 , bà tôi đã mang về một lá thư từ một đứa con bò sát. ',
 ' Đọc , " Khi bạn đọc điều này , tất cả các thành viên gia đình sẽ không tồn tại trong thế giới này , bởi vì chúng tôi đã không ăn thịt trong quá khứ 2 tuần

In [ ]:
! pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sacrebleu.metrics import BLEU
def calc_bleu(targets, translations, case_sensitive = False):
  if case_sensitive:
    print("Calculating case sensitive BLEU")
    targets = [sentence.strip() for sentence in targets]
    sys = [translation.strip() for translation in translations] 
  else:
    print("Calculating case insensitive BLEU")
    targets = [sentence.strip().lower() for sentence in targets]
    sys = [translation.strip().lower() for translation in translations] 

  refs = [targets]
  bleu = BLEU()
  a = bleu.corpus_score(sys, refs)
  print(a)

In [ ]:
calc_bleu(vi_test_sents, translations, case_sensitive = False)
calc_bleu(vi_test_sents, translations, case_sensitive = True)

Calculating case insensitive BLEU
BLEU = 25.57 58.7/32.9/19.4/11.8 (BP = 0.992 ratio = 0.992 hyp_len = 33444 ref_len = 33727)
Calculating case sensitive BLEU
BLEU = 24.80 57.5/31.9/18.8/11.3 (BP = 0.992 ratio = 0.992 hyp_len = 33444 ref_len = 33727)
